In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os, sys

In [3]:
import pylab as plt
import matplotlib

import numpy as np
import pandas as pd

In [4]:
sys.path.append('..')

from pyWebOmics.constants import HOMO_SAPIENS, REACTIONS, PROTEOMICS, METABOLOMICS, GENOMICS, TRANSCRIPTOMICS, PATHWAYS
from pyWebOmics.mapping import Mapper
from pyWebOmics.common import set_log_level_info, set_log_level_debug

2021-01-21 17:52:57.365 | INFO     | pyWebOmics.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# Demonstration of pyWebOmics

## Load the processed Covid data from [1]

[1] [Shen, Bo, et al. "Proteomic and metabolomic characterization of COVID-19 patient sera." Cell 182.1 (2020): 59-72.](https://www.sciencedirect.com/science/article/pii/S0092867420306279?casa_token=wKwWn9P4MK0AAAAA:v8z5MVnQ1ONrcatncCsigSDoxeOq2ZOcN4da9SofGDcpeDqrC76QK8yNKrKtVUrMWBBAntI8)

In [5]:
DATA_FOLDER = os.path.abspath(os.path.join('test_data', 'covid19_data'))
DATA_FOLDER

'C:\\Users\\joewa\\Work\\git\\pyWebOmics\\notebooks\\test_data\\covid19_data'

Read proteomics data

In [6]:
protein_data = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_data.csv'), index_col='Identifier')
protein_design = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_design.csv'), index_col='sample')

In [7]:
protein_data.head()

,h_F1_131N,h_F1_131C,h_F1_132C,h_F2_131N,h_F2_131C,h_F2_132C,h_F3_131N,h_F3_131C,h_F3_132C,h_F4_131N,...,s_F3_128N,s_F3_128C,s_F3_129C,s_F4_128N,s_F4_128C,s_F5_128N,s_F5_128C,s_F6_128N,s_F6_128C,s_F6_133N
Identifier,,,,,,,,,,,,,,,,,,,,,
P04114,0.750,0.853,0.822,1.191,1.175,1.078,0.693,0.947,0.931,1.057,...,1.044,1.305,1.657,1.323,1.624,1.170,0.981,0.791,1.029,1.195
P01024,0.782,1.057,0.994,0.864,0.917,0.790,0.823,1.152,0.816,0.920,...,1.100,0.986,1.114,1.210,1.289,1.104,1.111,1.007,1.159,0.979
P02768,1.183,1.101,1.045,1.086,1.041,1.187,1.234,1.079,1.011,1.099,...,0.786,0.706,0.947,0.831,0.717,0.795,0.776,0.938,0.903,0.743
P01023,1.066,1.278,0.959,0.811,0.789,0.931,0.971,0.769,1.011,0.866,...,0.817,0.728,0.861,0.798,0.751,0.917,0.809,0.780,1.195,0.706
P02751,1.085,0.947,0.993,1.343,1.130,0.778,0.731,1.084,1.107,0.909,...,0.566,0.854,1.109,0.630,0.850,0.661,0.848,0.829,0.760,0.811


In [8]:
protein_design

,group
sample,
h_F1_131N,healthy
h_F1_131C,healthy
h_F1_132C,healthy
h_F2_131N,healthy
h_F2_131C,healthy
...,...
s_F5_128N,severe
s_F5_128C,severe
s_F6_128N,severe


Read metabolomics data

In [9]:
compound_data = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_data.csv'), index_col='Identifier')
compound_design = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_design.csv'), index_col='sample')

In [10]:
compound_data.head()

,h_jkdz1,h_jkdz2,h_jkdz3,h_jkdz4,h_jkdz5,h_jkdz6,h_jkdz7,h_jkdz8,h_jkdz9,h_jkdz10,...,s_ZX12,s_ZX13,s_ZX14,s_ZX15,s_ZX16,s_ZX17,s_ZX18,s_ZX19,s_ZX20,s_ZX21
Identifier,,,,,,,,,,,,,,,,,,,,,
C21482,19413052.00,6.381812e+06,9.748316e+06,5.326872e+06,1.998072e+07,3.580375e+06,8.256121e+06,8.079382e+06,1.559659e+07,1.520363e+07,...,1.904349e+06,3.226016e+06,7.378147e+05,2.817698e+06,3.329101e+06,3206752.75,1.466174e+06,2.779301e+06,2.117668e+06,2.184310e+06
C18218,2711915.25,2.056393e+06,1.445594e+06,2.038765e+06,2.536996e+06,2.638198e+06,2.285757e+06,1.973140e+06,2.015425e+06,2.290842e+06,...,1.409720e+06,1.413307e+06,3.218834e+06,1.602131e+06,1.317878e+06,2930312.75,1.168094e+06,2.946776e+06,1.417311e+06,1.474166e+06
C05127,87727.25,NaN,9.238706e+04,NaN,1.597879e+05,NaN,NaN,NaN,9.055130e+04,1.214114e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.382788e+05,NaN,NaN
C01152,58832828.00,5.843934e+07,5.552133e+07,4.516214e+07,5.478952e+07,3.941259e+07,2.987876e+07,6.751726e+07,4.666031e+07,9.118524e+07,...,2.881314e+07,3.164358e+07,2.538767e+07,3.307604e+07,3.915698e+07,24400592.00,2.593375e+07,6.413868e+07,4.020588e+07,4.904488e+07
C02918,NaN,1.815549e+05,2.240392e+05,1.609397e+05,3.206194e+05,7.176557e+05,3.268182e+05,5.135810e+05,2.734582e+05,NaN,...,3.337245e+05,NaN,4.347152e+05,3.532118e+04,NaN,655827.25,8.359706e+05,4.034381e+06,2.839358e+05,8.062160e+04


In [11]:
compound_design

,group
sample,
h_jkdz1,healthy
h_jkdz2,healthy
h_jkdz3,healthy
h_jkdz4,healthy
h_jkdz5,healthy
...,...
s_ZX17,severe
s_ZX18,severe
s_ZX19,severe


In [12]:
set_log_level_info()

1

## Create a mapping object

The mapping object uses Reactome to map the different biological entities in the data:
- Transcripts (or genes) are connected to the proteins they encode
- Proteins and compounds are connected to reactions they're involved in
- Reactions are connected to pathways

In [13]:
m = Mapper(HOMO_SAPIENS) \
        .set_protein(protein_data, protein_design) \
        .set_compound(compound_data, compound_design) \
        .build()

2021-01-21 17:53:00.414 | INFO     | pyWebOmics.mapping:build:78 - Created a multi-omics network with 6445 nodes and 13840 edges
2021-01-21 17:53:00.424 | INFO     | pyWebOmics.mapping:build:79 - node_counts = {'Genomics': 0, 'Transcriptomics': 0, 'Proteomics': 594, 'Metabolomics': 81, 'Reactions': 5770, 'Pathways': 0}


In [14]:
m

## Query mapping object

Below shows some example queries we can perform with the mapping object

##### Find out the number of entities mapped to each reaction

In [15]:
reactions = m.get_nodes(types=REACTIONS)
count = 0
for reaction_id, reaction_data in reactions:
    reaction_name = reaction_data['name']
    proteins = m.get_connected(reaction_id, REACTIONS, PROTEOMICS)
    compounds = m.get_connected(reaction_id, REACTIONS, METABOLOMICS)
    
    if len(proteins) > 0 and len(compounds) > 0:
        print('%s\t%s (num_proteins=%d num_compounds=%d)' % (reaction_id, reaction_name, len(proteins), len(compounds)))
        count += 1
        if count >= 10:
            break

R-HSA-8952289	FAM20C phosphorylates FAM20C substrates (num_proteins=50 num_compounds=1)
R-HSA-749448	Liganded Gq-activating GPCRs bind inactive heterotrimeric Gq (num_proteins=6 num_compounds=9)
R-HSA-749452	The Ligand:GPCR:Gq complex dissociates (num_proteins=6 num_compounds=8)
R-HSA-379048	Liganded Gq/11-activating GPCRs act as GEFs for Gq/11 (num_proteins=6 num_compounds=9)
R-HSA-749454	The Ligand:GPCR:Gi complex dissociates (num_proteins=10 num_compounds=5)
R-HSA-749456	Liganded Gi-activating GPCRs bind inactive heterotrimeric G-protein Gi (num_proteins=10 num_compounds=6)
R-HSA-380073	Liganded Gi-activating GPCR acts as a GEF for Gi (num_proteins=10 num_compounds=6)
R-HSA-9606162	Phosphorylated BTK phosphorylates PLCG2 (num_proteins=45 num_compounds=1)
R-HSA-9606159	BTK autophosphorylates (num_proteins=45 num_compounds=1)
R-HSA-9606163	p-SYK and LYN phosphorylate BTK (num_proteins=45 num_compounds=1)


##### List all entities connected to reaction R-HSA-749448

In [16]:
query_id = 'R-HSA-749448'
m.get_neighbours(query_id)

['P01042',
 'P0DJI8',
 'P00734',
 'P01019',
 'Q9HAV0',
 'P05067',
 'C00002',
 'C00035',
 'C00025',
 'C00015',
 'C02165',
 'C00780',
 'C00047',
 'C00062',
 'C00077']

##### Query the connections between proteins and compounds (through their shared reactions)

In [17]:
query_id = 'P01042'
m.get_connected(query_id, PROTEOMICS, METABOLOMICS)

['C00035',
 'C00015',
 'C00042',
 'C00002',
 'C00025',
 'C02165',
 'C00186',
 'C00047',
 'C00062',
 'C00077',
 'C00029',
 'C00780']

In [18]:
query_id = 'C00077'
m.get_connected(query_id, METABOLOMICS, PROTEOMICS)

['P01019', 'P0DJI8', 'P01042', 'Q9HAV0', 'P00734', 'P05067']